In [1055]:
import numpy as np
import time
import cv2

In [1056]:
# Modifiable inputs: matrix, V, p, q, method
# matrix: a 2-dimension matrix
# V: an array of donated vertex 
# p: position of start pixel
# q: position of end pixel
# method: '4'=4-adjacency; '8'=8-adjacency; 'm'=m-adjacency; 'all'=run all methods by sequence
# print_all: set to True to print shortest distance to all pixels from p
#            set to False to print only the desired shortest distance from p to q
matrix = np.array([[3,1,2,1],
                   [2,2,0,2],
                   [1,2,1,1],
                   [1,0,1,2]])
V = np.array([2,1])
p = np.array([3,0])
q = np.array([0,3])
method = 'all'
print_all = False

In [1057]:
# test case
image = np.random.randint(3,size=(50,50))
V_ = np.array([2,1])
p_ = np.array([49,0])
q_ = np.array([0,49])
method_ = 'all'

In [1058]:
# convert 2-d position to 1-d position
def convert_index(col, i, j):
    if i == 0:
        index = j
    else:
        index = i*col+j
    
    return index

# convert 1-d position to 2-d position
def convert_row_col(col, index):
    i = int(index/col)
    j = np.mod(index,col)
    
    return i, j

In [1059]:
# print shortest path from p to the desired pixels by sequence
def print_path(col, current_vertex, parents):
    if current_vertex == -1:
        return
    print_path(col, parents[current_vertex], parents)
    
    i, j = convert_row_col(col, current_vertex)
    print("(%d, %d)" % (i, j), end=" ")

In [1060]:
# define a class of Graph to represent size, position, and relationships of pixels
class Graph:
    def __init__(self, size):
        self.adj_matrix = [[0] * size for _ in range(size)]
        self.size = size
        self.vertex_data = [''] * size
        self.parents = [-1] * self.size

    def add_edge(self, u, v):
        if 0 <= u < self.size and 0 <= v < self.size:
            self.adj_matrix[u][v] = 1
            self.adj_matrix[v][u] = 1

    def add_vertex_data(self, vertex, data):
        if 0 <= vertex < self.size:
            self.vertex_data[vertex] = data
    
    def dijkstra(self, start_vertex):
        distances = [float('inf')] * self.size
        distances[start_vertex] = 0
        visited = [False] * self.size

        for _ in range(self.size):
            min_distance = float('inf')
            u = None
            for i in range(self.size):
                if not visited[i] and distances[i] < min_distance:
                    min_distance = distances[i]
                    u = i

            if u is None:
                break

            visited[u] = True

            for v in range(self.size):
                if self.adj_matrix[u][v] > 0 and not visited[v]:
                    alt = distances[u] + self.adj_matrix[u][v]
                    if alt < distances[v]:
                        distances[v] = alt
                        self.parents[v] = u

        return distances

In [1061]:
# route paths for pixels based on 4-adjacency
def matrix_to_4_adj(matrix, V):
    row = len(matrix)
    col = len(matrix[0])
    g = Graph(row*col)
    
    for i in range(row):
        for j in range(col):
            g.add_vertex_data(convert_index(col,i,j), convert_index(col,i,j))
            if matrix[i][j] in V:
                
                if matrix[i-1][j] in V:
                    g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j))
                if j != 0:
                    if matrix[i][j-1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i,j-1))
    
    return g

In [1062]:
def run_4_adj(matrix, V, p, q):
    col = len(matrix[0])
    g4 = matrix_to_4_adj(matrix, V)
    distance_4 = g4.dijkstra(convert_index(col, p[0], p[1]))
    answer = distance_4[convert_index(col, q[0], q[1])]
    if (answer == float('inf')):
        print("\nThe path for 4-adjacency does not exist\n")
    else:
        print("\nThe shortest distance from p to q in 4-adjacency is:", distance_4[convert_index(col, q[0], q[1])], end=" ")
        print("   Path:", end=" ")
        print_path(col, convert_index(col, q[0], q[1]), g4.parents)
        print("\n")

    if print_all:
        print("4-adjacency shortest distance results:")
        for i, d in enumerate(distance_4):
            print(f"Distance from p to ({int(g4.vertex_data[i]/col)}, {np.mod(g4.vertex_data[i],col)}): {d}", end=" ")
            if i != convert_index(col, p[0], p[1]) and d != float('inf'):
                print("   Path:", end=" ")
                print_path(col, g4.vertex_data[i], g4.parents)
            print("\n")

In [1063]:
# route paths for pixels based on 8-adjacency
def matrix_to_8_adj(matrix, V):
    row = len(matrix)
    col = len(matrix[0])
    g = Graph(row*col)
    
    for i in range(row):
        for j in range(col):
            g.add_vertex_data(convert_index(col,i,j), convert_index(col,i,j))
            if matrix[i][j] in V:
                
                # 4-adjacency
                if matrix[i-1][j] in V:
                    g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j))
                if j != 0:
                    if matrix[i][j-1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i,j-1))
                
                # Diagonal-adjacency
                if (i != 0) & (j == 0):
                    if matrix[i-1][j+1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j+1))
                if (i != 0) & (j == col-1):
                    if matrix[i-1][j-1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j-1))
                if (i != 0) & (j != 0) & (j != col-1):
                    if matrix[i-1][j-1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j-1))
                    if matrix[i-1][j+1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j+1))
    
    return g

In [1064]:
def run_8_adj(matrix, V, p, q):
    col = len(matrix[0])
    g8 = matrix_to_8_adj(matrix, V)
    distance_8 = g8.dijkstra(convert_index(col, p[0], p[1]))
    answer = distance_8[convert_index(col, q[0], q[1])]
    if (answer == float('inf')):
        print("\nThe path for 8-adjacency does not exist\n")
    else:
        print("\nThe shortest distance from p to q in 8-adjacency is:", distance_8[convert_index(col, q[0], q[1])], end=" ")
        print("   Path:", end=" ")
        print_path(col, convert_index(col, q[0], q[1]), g8.parents)
        print("\n")

    if print_all:
        print("8-adjacency shortest distance results:")
        for i, d in enumerate(distance_8):
            print(f"Distance from p to ({int(g8.vertex_data[i]/col)}, {np.mod(g8.vertex_data[i],col)}): {d}", end=" ")
            if i != convert_index(col, p[0], p[1]) and d != float('inf'):
                print("   Path:", end=" ")
                print_path(col, g8.vertex_data[i], g8.parents)
            print("\n")

In [1065]:
# route paths for pixels based on m-adjacency
def matrix_to_m_adj(matrix, V):
    row = len(matrix)
    col = len(matrix[0])
    g = Graph(row*col)
    
    for i in range(row):
        for j in range(col):
            g.add_vertex_data(convert_index(col,i,j), convert_index(col,i,j))
            if matrix[i][j] in V:
                
                # 4-adjacency
                if matrix[i-1][j] in V:
                    g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j))
                if j != 0:
                    if matrix[i][j-1] in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i,j-1))
                
                # Diagonal-adjacency with condition
                if (i != 0) & (j == 0):
                    if matrix[i-1][j+1] in V and matrix[i-1][j] not in V and matrix[i][j+1] not in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j+1))
                if (i != 0) & (j == col-1):
                    if matrix[i-1][j-1] in V and matrix[i-1][j] not in V and matrix[i][j-1] not in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j-1))
                if (i != 0) & (j != 0) & (j != col-1):
                    if matrix[i-1][j-1] in V and matrix[i-1][j] not in V and matrix[i][j-1] not in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j-1))
                    if matrix[i-1][j+1] in V and matrix[i-1][j] not in V and matrix[i][j+1] not in V:
                        g.add_edge(convert_index(col,i,j), convert_index(col,i-1,j+1))
    
    return g

In [1066]:
def run_m_adj(matrix, V, p, q):
    col = len(matrix[0])
    gm = matrix_to_m_adj(matrix, V)
    distance_m = gm.dijkstra(convert_index(col, p[0], p[1]))
    answer = distance_m[convert_index(col, q[0], q[1])]
    if (answer == float('inf')):
        print("\nThe path for m-adjacency does not exist\n")
    else:
        print("\nThe shortest distance from p to q in m-adjacency is:", distance_m[convert_index(col, q[0], q[1])], end=" ")
        print("   Path:", end=" ")
        print_path(col, convert_index(col, q[0], q[1]), gm.parents)
        print("\n")

    if print_all:
        print("m-adjacency shortest distance results:")
        for i, d in enumerate(distance_m):
            print(f"Distance from p to ({int(gm.vertex_data[i]/col)}, {np.mod(gm.vertex_data[i],col)}): {d}", end=" ")
            if i != convert_index(col, p[0], p[1]) and d != float('inf'):
                print("   Path:", end=" ")
                print_path(col, gm.vertex_data[i], gm.parents)
            print("\n")

In [1067]:
def run_methods(matrix, V, p, q, method):
    print("matrix =\n", matrix)
    print("V =", V)
    print("p =", p)
    print("q =", q)
    print("method =", method)
    print("--------------------------------------------------------------------------------")
    if (method == '4'):
        start = time.time()
        run_4_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime = %.9f seconds" % (end-start))
    elif (method == '8'):
        start = time.time()
        run_8_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime = %.9f seconds" % (end-start))
    elif (method == 'm'):
        start = time.time()
        run_m_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime = %.9f seconds" % (end-start))
    elif (method == 'all'):
        start = time.time()
        run_4_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime for 4-adjacency = %.9f seconds" % (end-start))
        print("--------------------------------------------------------------------------------")
        start = time.time()
        run_8_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime for 8-adjacency = %.9f seconds" % (end-start))
        print("--------------------------------------------------------------------------------")
        start = time.time()
        run_m_adj(matrix, V, p, q)
        end = time.time()
        print("Runtime for m-adjacency = %.9f seconds" % (end-start))

In [1068]:
# test run
run_methods(image, V_, p_, q_, method_)

matrix =
 [[0 0 0 ... 0 0 1]
 [2 2 1 ... 2 0 1]
 [0 2 0 ... 2 1 1]
 ...
 [0 1 2 ... 0 0 0]
 [1 0 0 ... 1 1 0]
 [1 2 1 ... 0 2 2]]
V = [2 1]
p = [49  0]
q = [ 0 49]
method = all
--------------------------------------------------------------------------------

The shortest distance from p to q in 4-adjacency is: 110    Path: (49, 0) (49, 1) (49, 2) (49, 3) (48, 3) (47, 3) (47, 2) (46, 2) (45, 2) (44, 2) (44, 3) (43, 3) (42, 3) (42, 4) (41, 4) (40, 4) (40, 5) (40, 6) (40, 7) (40, 8) (40, 9) (39, 9) (39, 10) (39, 11) (39, 12) (39, 13) (38, 13) (38, 14) (37, 14) (36, 14) (36, 15) (36, 16) (35, 16) (34, 16) (33, 16) (33, 17) (32, 17) (31, 17) (31, 16) (30, 16) (29, 16) (28, 16) (27, 16) (26, 16) (25, 16) (24, 16) (23, 16) (23, 15) (22, 15) (21, 15) (21, 14) (21, 13) (20, 13) (19, 13) (18, 13) (17, 13) (16, 13) (16, 14) (16, 15) (16, 16) (16, 17) (16, 18) (16, 19) (15, 19) (14, 19) (14, 20) (13, 20) (13, 21) (13, 22) (13, 23) (13, 24) (13, 25) (13, 26) (12, 26) (12, 27) (12, 28) (12, 29) (12,

In [1069]:
# main function
# please modify input arguments to match format and requirements
def main(argc, argv):
    if argc != 6:
        print("Incorrect input format")
    else:
        print(argv[0][0])
        file = argv[1][0]
        # image = cv2.imread(file, cv2.IMREAD_UNCHANGED)
        V = argv[2][0]
        p = argv[3][0]
        q = argv[4][0]
        method = argv[5][0]
        
        run_methods(file, V, p, q, method)

argc = 6
# argv = np.array([["p2_c.py"], ["wolves.png"], [V], [p], [q], [method]])
argv = np.array([["p2_c.py"], [matrix], [V], [p], [q], [method]])
main(argc, argv)

p2_c.py
matrix =
 [[3 1 2 1]
 [2 2 0 2]
 [1 2 1 1]
 [1 0 1 2]]
V = [2 1]
p = [3 0]
q = [0 3]
method = all
--------------------------------------------------------------------------------

The shortest distance from p to q in 4-adjacency is: 6    Path: (3, 0) (2, 0) (1, 0) (1, 1) (0, 1) (0, 2) (0, 3) 

Runtime for 4-adjacency = 0.000000000 seconds
--------------------------------------------------------------------------------

The shortest distance from p to q in 8-adjacency is: 4    Path: (3, 0) (2, 0) (1, 1) (0, 2) (0, 3) 

Runtime for 8-adjacency = 0.000000000 seconds
--------------------------------------------------------------------------------

The shortest distance from p to q in m-adjacency is: 6    Path: (3, 0) (2, 0) (1, 0) (1, 1) (0, 1) (0, 2) (0, 3) 

Runtime for m-adjacency = 0.000996590 seconds


c:\Users\gaoyz\anaconda3\envs\ece558\lib\site-packages\ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
